In [1]:
import pandas as pd
import numpy as np
agora_data = pd.read_json('agora_hack_products.json')
agora_data

,product_id,name,props,is_reference,reference_id
0,0007302f2fe1d54d,Классическая сплит-система ROYAL CLIMA PANDORA...,"[Класс энергоэффективности\tA, Мощность конд...",False,f497219eb0077f84
1,000740b6c1cc763e,Смартфон Xiaomi Redmi Note 10S NFC 6/128 ГБ RU...,"[Экран\t6.43"" (2400x1080) AMOLED 60 Гц, 4 каме...",True,None
2,0039af5efceac4ab,Холодильник Бирюса 118,[Мощность замораживания 4 кг/сутки],False,28085e941cde1639
3,004f2158acb8165c,ASUS TUF-GTX1660S-O6G-GAMING Видеокарта,"[Объем видеопамяти\t6144 МБ, Частота памяти\...",False,9afe55bb4bf1e8a8
4,005cddb29e1677ec,"Кофемашина Saeco Lirika One Touch Cappuccino, ...","[Приготовление капучино\tавтоматическое, Мате...",False,1f21918ceb5d345c
...,...,...,...,...,...
3246,ff5da4be6fa60c4b,"15.6"" Ноутбук Lenovo IdeaPad Gaming 315IHU6 19...","[Видеокарта\tNVIDIA GeForce RTX 3050 4 ГБ, Вре...",True,None
3247,ff66532467a02652,Моющий робот-пылесос для дома и квартиры Xiaom...,"[Объем контейнера для воды\t0.27 л, ШхГхВ\t35....",False,d4ebc4a26700d5e0
3248,ff75ade409f4da7e,"Huawei Умный браслет Band 7, графитово-черный","[Модификация\tGPS, Материал корпуса\tпластик]",False,177ccb3b84125efa
3249,ffb770de0c2feafc,Видеокарта Gigabyte GV-R675XTGAMING OC-12GD 12...,[Комплектация\tRetail],False,a07d5538ebec8e36


## деление датасета на эталоны и товары

In [2]:
agora_data_goods = agora_data[agora_data['is_reference'] == False]
agora_data_prime = agora_data[agora_data['is_reference'] == True]

## токенизация имён

In [3]:
import codecs
f = codecs.open('russian_colors.txt', 'r', "utf-8")
rus_words = [line.strip() for line in f]
f1 = open('english_colors.txt', 'r')
eng_words = [line.strip() for line in f1]

colors = dict(zip(eng_words, rus_words))
colors

{'alizarin': 'ализарин',
 'amaranth': 'амарант',
 'amber': 'янтарь',
 'amethyst': 'аметист',
 'apricot': 'абрикос',
 'aqua': 'цвет морской волны',
 'aquamarine': 'Аквамарин',
 'asparagus': 'спаржа',
 'auburn': 'темно-рыжий',
 'azure': 'лазурь',
 'beige': 'бежевый',
 'bistre': 'бистр',
 'black': 'черный',
 'blue': 'синий',
 'blue-green': 'цвет морской волны',
 'blue-violet': 'сине-фиолетовый',
 'bondi-blue': 'бонди-синий',
 'brass': 'латунь',
 'bronze': 'бронза',
 'brown': 'коричневый',
 'buff': 'бафф',
 'burgundy': 'Бургундия',
 'camouflage-green': 'камуфляжно-зеленый',
 'caput-mortuum': 'головной убор',
 'cardinal': 'кардинал',
 'carmine': 'кармин',
 'carrot-orange': 'морковно-апельсиновый',
 'celadon': 'селадон',
 'cerise': 'вишневый',
 'cerulean': 'лазурный',
 'champagne': 'шампанское',
 'charcoal': 'древесный уголь',
 'chartreuse': 'шартрез',
 'cherry-blossom-pink': 'вишнево-розовый',
 'chestnut': 'каштан',
 'chocolate': 'шоколад',
 'cinnabar': 'киноварь',
 'cinnamon': 'корица',
 '

In [4]:
import re
reg = re.compile('[^a-zа-я0-9 ]')

def make_tokens(input_str):
    tokens = input_str.lower().replace("/", " ").replace("-", " ").replace("\t", " ").replace("pro", " pro")
    tokens = reg.sub('', tokens).replace("ё", "е").replace("ghz", "ггц").replace("gb", "гб")
    tokens = tokens.split(" ")
    new_tokens = []
    for i in tokens:
        if i in colors.keys():
            i = colors[i]
        new_tokens += re.findall(r'[a-zа-я]+', i)
        new_tokens += re.findall(r'\d+', i)
    return new_tokens

In [5]:
Xname = {}
Xprops = []
for index, row in agora_data_goods.iterrows():
    tmp = make_tokens(row['name'])
    Xname[agora_data_goods.product_id[index]] = set(tmp)
    tmp = make_tokens(' '.join(row['props']))
    Xprops.append(set(tmp))

etalonsname = {}
etalonsprops = []
for index, row in agora_data_prime.iterrows():
    tmp = make_tokens(row['name'])
    etalonsname[agora_data_prime.product_id[index]] = set(tmp)
    tmp = make_tokens(' '.join(row['props']))
    etalonsprops.append(set(tmp))

y = np.array(agora_data_goods.reference_id)
etalonsname

{'000740b6c1cc763e': {'10',
  '128',
  '6',
  'nfc',
  'note',
  'redmi',
  'ru',
  's',
  'xiaomi',
  'белоснежная',
  'галька',
  'гб',
  'смартфон'},
 '0083737f904dd9a9': {'1058', 'lu', 'lumme', 'фен'},
 '01378fa92f901df5': {'05',
  '1080',
  '15',
  '156',
  '1920',
  '23',
  '256',
  '3',
  '40033',
  '4500',
  '5',
  '81',
  'amd',
  'are',
  'dos',
  'graphics',
  'ideapad',
  'lenovo',
  'radeon',
  'rk',
  'ryzen',
  'ssd',
  'u',
  'w',
  'x',
  'гб',
  'ггц',
  'ноутбук',
  'платина',
  'серый'},
 '01601abb0bf5133d': {'2071',
  'd',
  'hym',
  'hyundai',
  'микроволновая',
  'печь',
  'серебристый'},
 '01a28e17323d6ef8': {'3020', 'bi', 'phaser', 'xerox', 'лазерный', 'принтер'},
 '01e52344308143f6': {'02',
  '4',
  'g',
  'weissgauff',
  'wgs',
  'ws',
  'белый',
  'газовая',
  'плита'},
 '0229abcf2ded1058': {'525', 'caferomatica', 'nicr', 'nivona', 'кофемашина'},
 '05a95e7066dc4c52': {'7',
  'band',
  'global',
  'mi',
  'xiaomi',
  'браслет',
  'умный',
  'черный'},
 '05cb6

Сравниваем токены у товаров и эталонов

In [6]:
ans = []
for i, j in zip(Xname.values(), Xprops):
    comp_name = 0 # число совпадений в имени
    comp_props = 0 # число совпадений в свойствах
    tmp_ans = 0
    for k, l, m in zip(etalonsname.keys(), etalonsname.values(), etalonsprops):
        # если общее число совпадений у двух эталонов равняется
        if len(i & l) + len(m & j) == comp_name + comp_props:
            # выбираем эталон, с которым совпало больше свойств
            if len(m & j) > comp_props:
                comp_name = len(i & l)
                comp_props = len(m & j)
                tmp_ans = k
        if (len(i & l) + len(m & j) > comp_name + comp_props):
            comp_name = len(i & l)
            comp_props = len(m & j)
            tmp_ans = k
    ans.append(tmp_ans)

In [7]:
from sklearn.metrics import accuracy_score
accuracy_score(y, np.array(ans))

0.9748201438848921

In [10]:
df = pd.DataFrame({'true': y, 'pred': np.array(ans), 'prod_id': np.array(Xname.keys())})
df_wrong = df.where(df.true != df.pred).dropna()
df_wrong

,true,pred,prod_id
4,8f9c8fcf9cb2862c,647995c157f7d99a,0083dddef6bc6503
16,516c4c0cca619ea4,d4ebc4a26700d5e0,01613fb413f4ccce
31,be85c7f8554f16c0,80abb5aed92419c1,028be720c37aae99
75,11777853e7c22618,ff5da4be6fa60c4b,05839985429a5c5a
89,1253c3296a3c0153,13f78f44e27fec08,06cf3e4f54402e5e
...,...,...,...
2519,ea2228931c428185,177ccb3b84125efa,e731a4972090a642
2583,6d65b7363fdef730,0faf2717bf811454,edecb3e6279c8029
2604,b74de9b0b28590b4,3914651e72846423,eff30f1c224b36dc
2618,e09b76f558f1befe,e043872d672a8fe8,f0e1c5edb53074f6


In [11]:
df_wrong2 = pd.DataFrame({'prod_name': [], 'prod_props': [], 'true_ref_name': [], 'true_ref_props': [], 'pred_ref_name': [], 'pred_ref_props': []})
data_new = agora_data.set_index('product_id')
for index, row in df_wrong.iterrows():
    new_row = { 'prod_name': data_new.name[row.prod_id],
                      'prod_props': data_new.props[row.prod_id],
                      'true_ref_name': data_new.name[row.true],
                      'true_ref_props': data_new.props[row.true],
                      'pred_ref_name': data_new.name[row.pred],
                      'pred_ref_props': data_new.props[row.pred]}
    df_wrong2 = df_wrong2.append(new_row, ignore_index=True)
df_wrong2

C:\Users\Андрей\AppData\Local\Temp\ipykernel_13212\1947823895.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wrong2 = df_wrong2.append(new_row, ignore_index=True)
C:\Users\Андрей\AppData\Local\Temp\ipykernel_13212\1947823895.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wrong2 = df_wrong2.append(new_row, ignore_index=True)
C:\Users\Андрей\AppData\Local\Temp\ipykernel_13212\1947823895.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wrong2 = df_wrong2.append(new_row, ignore_index=True)
C:\Users\Андрей\AppData\Local\Temp\ipykernel_13212\1947823895.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wrong2 = d

,prod_name,prod_props,true_ref_name,true_ref_props,pred_ref_name,pred_ref_props
0,Lenovo 62A8KAT1EU Монитор,[Яркость\t250 кд/м²; контрастность: 1000:1],"23.8"" Монитор Lenovo C24-20, 1920x1080, *VA, ч...","[Тип матрицы\t*VA, Яркость\t250 кд/м²; контрас...",Монитор Lenovo ThinkVision E20-20 (62BBKAT1EU),"[Макс. разрешение\t1440x900, Покрытие экрана\t..."
1,Xiaomi Mi Robot Vacuum Mop 2 Pro White [BHR504...,"[Объем контейнера для пыли\t0.45 л, Работа о...",Mi Robot Vacuum-Mop 2 Pro,"[Тип уборки\tсухая и влажная, Объем контейнера...",Робот-пылесос Xiaomi Mi Robot Vacuum-Mop 2 Lit...,"[Тип уборки\tсухая и влажная, Объем контейнера..."
2,"ASUS Монитор 24"" ASUS VA24DQLB (90LM0541-B01370)","[Тип матрицы\tIPS, Экран\t1920x1080 (16:9)...","23.8"" Монитор ASUS VA24DQLB, 1920x1080, 75 Гц,...","[Яркость\t250 кд/м²; контрастность: 1000:1, Ин...",Монитор ASUS VA24DQLB-W (90LM0544-B01370),"[Соотношение сторон\t16:9, Цвет товара\tчерн..."
3,Ноутбук Lenovo 15ACH6 82K201D1RK Ryzen 5 5600H...,[Видеопамять\t4 ГБ],Ноутбук Lenovo IP Gaming 3 15ACH6 (82K201D1RK),"[Накопитель\t256 ГБ, Графика\tNVIDIA GeForce R...","15.6"" Ноутбук Lenovo IdeaPad Gaming 315IHU6 19...","[Видеокарта\tNVIDIA GeForce RTX 3050 4 ГБ, Вре..."
4,"Телевизор Xiaomi Mi TV 65"" L65M5-5ASP","[Разрешение\t3840x2160, Беспроводная связь Bl...","65"" Телевизор Xiaomi Mi TV 4S 65 T2S 2020 LED,...","[Диагональ\t65"", Частота обновления экрана\t60...","65"" Телевизор Xiaomi Mi TV 5 65 Pro 2019 QLED,...","[Диагональ\t65"", Форматы HDR\tHDR10+, Разрешен..."
...,...,...,...,...,...,...
65,Умный браслет Huawei Band 6 FRA-B19 Sakura Pin...,"[Емкость аккумулятора\t180 мА·ч, Совместимост...","Умный браслет HUAWEI Band 6, розовая сакура","[Экран\t1.47"" (198x368) AMOLED, Защищенность\t...",Умный браслет HUAWEI Band 7 (LEA-B19) графитов...,"[Модификация\tGPS, Материал корпуса\tпластик, ..."
66,Bosch wr 10-2p,"[Индикация\tвключения, Тип камеры сгорания\tот...",Проточный газовый водонагреватель Bosch WR 10-...,"[Тип камеры сгорания\tоткрытый, Управление\tме...",Проточный газовый водонагреватель Bosch WR 10-...,"[Полезная мощность\t17.4 кВт, Размеры (ШxВxГ)\..."
67,Стиральная машина встраиваемая Bosch WIW28540OE,[Скорость отжима\t1400 об/мин],"Стиральная машина Bosch WIW 28540, белый","[Установка\tвстраиваемая, ШхГхВ\t60х57х82 см, ...",Встраиваемая стиральная машина Bosch WIW28541EU,"[Цвет\tбелый, Установка\tвстраиваемая, Загрузк..."
68,Видеокарта ASUS GeForce GT 1030 1266Mhz PCI-E ...,"[Максимальное разрешение\t4096x2160, Объем в...",Видеокарта ASUS GeForce GT 1030 Silent LP 2GB ...,"[Комплектация\tRetail, Область применения\tиг...",Видеокарта Asus PCI-E GT730-4H-SL-2GD5 NVIDIA ...,"[Тип памяти\tGDDR5, Поддержка технологий\tподд..."


In [12]:
df_wrong2.to_excel('wrong.xlsx')

PermissionError: [Errno 13] Permission denied: 'wrong.xlsx'